In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.bronze;
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.silver;
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.gold;

In [0]:
from pyspark.sql.functions import current_timestamp

# Read raw CSV data
oct_df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)

# Add ingestion metadata
bronze_df = oct_df.withColumn("ingestion_time", current_timestamp())

# Bronze Delta path
bronze_path = "/Volumes/workspace/ecommerce/bronze/ecommerce_events"

# Write to Bronze Delta table
bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(bronze_path)

In [0]:
# 1. Check row count
print("Bronze row count:", bronze_df.count())

# 2. Preview data
display(bronze_df.limit(10))

# 3. Check schema
bronze_df.printSchema()

Bronze row count: 42448764


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_time
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2026-01-14T16:40:22.444Z
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2026-01-14T16:40:22.444Z
2019-10-01T00:00:01.000Z,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2026-01-14T16:40:22.444Z
2019-10-01T00:00:01.000Z,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2026-01-14T16:40:22.444Z
2019-10-01T00:00:04.000Z,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2026-01-14T16:40:22.444Z
2019-10-01T00:00:05.000Z,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,2026-01-14T16:40:22.444Z
2019-10-01T00:00:08.000Z,view,17300353,2053013553853497655,null,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d,2026-01-14T16:40:22.444Z
2019-10-01T00:00:08.000Z,view,31500053,2053013558031024687,null,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6,2026-01-14T16:40:22.444Z
2019-10-01T00:00:10.000Z,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,2026-01-14T16:40:22.444Z
2019-10-01T00:00:11.000Z,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,2026-01-14T16:40:22.444Z


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- ingestion_time: timestamp (nullable = false)



In [0]:
from pyspark.sql.functions import col

# Read Bronze data
bronze_df = spark.read.format("delta").load(
    "/Volumes/workspace/ecommerce/bronze/ecommerce_events"
)

# Clean and validate data
silver_df = (
    bronze_df
    .filter(col("user_id").isNotNull())
    .filter(col("event_type").isin("view", "cart", "purchase"))
    .filter((col("price").isNull()) | (col("price") >= 0))
    .dropDuplicates()
)

# Silver Delta path
silver_path = "/Volumes/workspace/ecommerce/silver/ecommerce_events_clean"

# Write to Silver Delta table
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_path)


In [0]:
# 1. Compare row counts
print("Bronze rows:", bronze_df.count())
print("Silver rows:", silver_df.count())

# 2. Check invalid user_id
print("Null user_id count:",
      silver_df.filter(col("user_id").isNull()).count())

# 3. Check invalid event types
silver_df.groupBy("event_type").count().show()

# 4. Check negative prices
print("Negative price count:",
      silver_df.filter(col("price") < 0).count())

# 5. Preview cleaned data
display(silver_df.limit(10))
     

Bronze rows: 42448764
Silver rows: 42418544
Null user_id count: 0
+----------+--------+
|event_type|   count|
+----------+--------+
|  purchase|  742773|
|      cart|  898443|
|      view|40777328|
+----------+--------+

Negative price count: 0


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_time
2019-10-13T06:26:29.000Z,view,7004549,2053013560346280633,kids.carriage,rant,69.47,552084975,0eba87f6-ba9d-4095-9473-a24b01f4cffd,2026-01-14T16:39:35.770Z
2019-10-13T06:26:38.000Z,view,1307073,2053013558920217191,computers.notebook,acer,694.74,512582502,6b96b3ca-1149-443e-bd4f-1c52881318e0,2026-01-14T16:39:35.770Z
2019-10-13T06:28:03.000Z,view,1003317,2053013555631882655,electronics.smartphone,apple,952.15,538995380,b871bc41-e4d8-4cb1-9818-5d316e376fc8,2026-01-14T16:39:35.770Z
2019-10-13T06:28:22.000Z,view,1307187,2053013558920217191,computers.notebook,acer,874.93,516252211,4995e684-727d-49c2-a7e2-d85a139afc9e,2026-01-14T16:39:35.770Z
2019-10-13T06:28:28.000Z,view,1307344,2053013558920217191,computers.notebook,asus,540.53,532239414,f3785774-5cba-4274-93c5-3120ef8b8da4,2026-01-14T16:39:35.770Z
2019-10-13T06:28:40.000Z,view,1002544,2053013555631882655,electronics.smartphone,apple,460.51,559521274,d1f59fec-e722-49fe-abff-25902eb8a48c,2026-01-14T16:39:35.770Z
2019-10-13T06:26:37.000Z,view,1004767,2053013555631882655,electronics.smartphone,samsung,250.83,515798681,f54c4661-f8e4-47ef-8518-40b5f7fc389b,2026-01-14T16:39:35.770Z
2019-10-13T06:28:03.000Z,view,1005129,2053013555631882655,electronics.smartphone,apple,1428.33,559692953,6870eef4-cf3b-41c5-bcd6-102142713889,2026-01-14T16:39:35.770Z
2019-10-13T06:28:24.000Z,view,1003879,2053013555631882655,electronics.smartphone,huawei,771.94,515627556,8c491a00-6c6e-4886-bc3b-5d91fdccb607,2026-01-14T16:39:35.770Z
2019-10-13T06:26:02.000Z,view,2701589,2053013563911439225,appliances.kitchen.refrigerators,samsung,1002.15,514238834,9f048c01-b25e-4c7f-9d8e-50a2d75c04fd,2026-01-14T16:39:35.770Z


In [0]:
from pyspark.sql.functions import col, to_date, sum, count, countDistinct

# Read Silver data
silver_df = spark.read.format("delta").load(
    "/Volumes/workspace/ecommerce/silver/ecommerce_events_clean"
)

# Create business aggregates
gold_df = (
    silver_df
    .filter(col("event_type") == "purchase")
    .withColumn("event_date", to_date("event_time"))
    .groupBy("event_date")
    .agg(
        sum("price").alias("total_revenue"),
        count("*").alias("total_orders"),
        countDistinct("user_id").alias("unique_customers")
    )
)

# Gold Delta path
gold_path = "/Volumes/workspace/ecommerce/gold/daily_sales_metrics"

# Write to Gold Delta table
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(gold_path)

In [0]:
# 1. Preview metrics
display(gold_df.orderBy("event_date").limit(10))

# 2. Check for duplicate dates
print("Duplicate dates:",
      gold_df.count() - gold_df.select("event_date").distinct().count())

# 3. Basic sanity checks
gold_df.select(
    "event_date",
    "total_revenue",
    "total_orders",
    "unique_customers"
).summary().show()

event_date,total_revenue,total_orders,unique_customers
2019-10-01,6275579.060000003,19305,14064
2019-10-02,6213628.530000013,19469,13894
2019-10-03,6233782.980000023,19255,13722
2019-10-04,8623058.190000035,27039,19214
2019-10-05,7341094.460000014,23492,16734
2019-10-06,6737258.170000014,22169,16096
2019-10-07,6348189.059999998,21378,15438
2019-10-08,6819701.260000004,23071,16590
2019-10-09,6855326.050000003,22747,16417
2019-10-10,6665413.209999992,21992,15959


Duplicate dates: 0
+-------+-----------------+-----------------+------------------+
|summary|    total_revenue|     total_orders|  unique_customers|
+-------+-----------------+-----------------+------------------+
|  count|               31|               31|                31|
|   mean|7417200.407419354|23960.41935483871|17193.354838709678|
| stddev|957899.7414448275|3083.451159271524|2042.3665284564227|
|    min|6213628.530000013|            19255|             13722|
|    25%|6633882.149999988|            21378|             15438|
|    50%|7307691.569999975|            23883|             17200|
|    75%|7913671.440000011|            25849|             18385|
|    max|9747164.719999958|            31393|             21993|
+-------+-----------------+-----------------+------------------+

